# 로봇제어 크롤링

In [1]:
from tqdm.notebook import tqdm
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.request import Request
import urllib

### 1차 크롤링

In [2]:
company_list = []

for i in range(1,2):
    url = "https://job.incruit.com/jobdb_list/searchjob.asp?kw=%B7%CE%BA%BF%C1%A6%BE%EE&rgn2=18&rgn2=11&page={page_number}"        # 자신의 찾으려는 웹사이트 입력
    req = Request(url.format(page_number = urllib.parse.quote(str(i))))
    res = urlopen(req)
    soup = BeautifulSoup(res, "html.parser")
    
    element = soup.find_all('ul',class_='c_row')
    for each in element:
        cl_top = each.find('div',class_='cl_top')
        cell_mid = each.find('div',class_='cell_mid')

        company = []
        name =each.find('a',class_='cpname').string    # 회사이름
        title = cell_mid.find(class_='cl_top').get_text()   # 모집명
        link = cell_mid.find('a')['href']                  # 링크
        info = cell_mid.find(class_='cl_md').get_text()     # 경력 학력 주소 연봉
        field = cell_mid.find(class_='cl_btm').get_text()  # 분야

        company.append(name)
        company.append(title)
        company.append(link)
        company.append(info)
        company.append(field)
        company_list.append(company)

In [3]:
company_list

[['(주)잡뉴스솔로몬서치',
  '\n로봇 제어 엔지니어 (경력 10년이상) - IT 서비스 전문기업, 그룹 계열사\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2401110001206&src=gsw*etc',
  '\n경력10년↑ 대졸이상\n경기 성남시 정규직\n',
  '\n기계·금속·재료\n'],
 ['(주)잡뉴스솔로몬서치',
  '\nR&D, 제어 알고리즘 (대리-과장급) - 산업용 로봇 생산기업, 그룹계열사\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2310220000068&src=gsw*etc',
  '\n경력3년↑ 대졸이상\n경기 성남시 정규직\n',
  '\n응용프로그래머\n'],
 ['(주)피플케어코리아',
  '\n자동차 차체설비 전기제어 로봇팀 팀장(부장급) - 2명\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2401190001383&src=gsw*etc',
  '\n경력13~20년 대졸\n경기 시흥시 정규직\n',
  '\n전자·반도체, 기계·금속·재료\n'],
 ['메리텍산업',
  '\n자동화기기 전기제어 신입/경력(다관절로봇 티칭 우대) 모집\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2401040003022&src=gsw*etc',
  '\n신입/경력(연차무관) 고졸\n경기 안산시 정규직\n',
  '\n전자·반도체\n'],
 ['㈜에이트테크',
  '\n[IT/로봇] (주)에이트테크  AI, 로봇제어, 백엔드 개발 채용\n스크랩\n',
  'https://job.incruit.com/jobdb_info/jobpost.asp?job=2208310001588&src=gsw*etc',
  '\n신입/경력(연차무관) 대졸이상\n서울 서초구 외 정규직\n',
  '

In [5]:
import pandas as pd 
robot_df = pd.DataFrame(company_list, columns=['기업이름', '모집명','링크','info','담당업무'])

In [6]:
robot_df['고용형태'] = robot_df['info'].str.split('\n').str[1]
robot_df['연봉'] = robot_df['info'].str.split('\n').str[2]
robot_df['모집명'] = robot_df['모집명'].str.split('\n').str[1]
robot_df['담당업무'] = robot_df['담당업무'].str.replace('\n', '')
robot_df['경력'] = robot_df['고용형태'].str.split(' ').str[0]

In [7]:
import re
robot_df['연봉'] = robot_df['연봉'].apply(lambda x: ''.join(re.findall(r'월급[\s\S]*[\d,~]+|[\d,~]+', x)) if re.search(r'월급|[\d,~]', x) else None)

In [8]:
robot_df.drop(columns=['info','고용형태'], inplace=True)

### 키워드 삽입

In [12]:
cate_list = []
for i in range(1,45):
    cate_list.append('로봇제어')

In [13]:
robot_df['키워드'] = cate_list

### 2차 크롤링

In [14]:
from tqdm import tqdm 
from user_agent import generate_user_agent

address_list = []
link2_list = []

for idx, row in tqdm(robot_df.iterrows(), total=len(robot_df.index)):
    req = Request(url=row['링크'], headers={'User-Agent':generate_user_agent()})
    page = urlopen(req)
    soup = BeautifulSoup(page, 'html.parser')

    try:
        company = soup.find(class_='jcinfo_list last')
        address = company.get_text()
        address_list.append(address)
    except (AttributeError, IndexError):
        address_list.append(None)

    try:
        link = soup.find_all('div',class_='section_view_layout')[1]
        link2 = link.find('iframe',id='ifrmJobCont')['src']
        link2_list.append(link2)
    except (AttributeError, IndexError):
        link2_list.append(None)


100%|██████████| 44/44 [00:34<00:00,  1.26it/s]


In [15]:
robot_df['주소'] = address_list
robot_df['링크2'] = link2_list

In [17]:
import re

# 주소 열에서 주소라는 단어를 발견했을 때 이후의 문자열을 추출하는 함수
def extract_address(text):
    if isinstance(text, str):  # 문자열인 경우에만 처리
        match = re.search(r'주소\s*(.*)', text)
        return match.group(1) if match else None
    else:
        return None

# 주소 열에 적용하여 새로운 열 생성
robot_df['추출된_주소'] = robot_df['주소'].apply(extract_address)

# 결과 확인
print(robot_df[['주소', '추출된_주소']])

                                                   주소  \
0   복리후생 연차, 월차, 경조휴가, 국민연금, 의료보험, 산재보험, 고용보험, 사원아...   
1   복리후생 연차, 월차, 경조휴가, 국민연금, 의료보험, 산재보험, 고용보험, 사원아...   
2   복리후생 주5일근무, 연차, 월차, 국민연금, 의료보험, 산재보험, 고용보험, 인센...   
3        주소  (15418) 경기도 안산시 단원구 지원로115번길 23 (성곡동)...   
4        주소  (22689) 인천광역시 서구 정서진로 410 D동 230호 (오류...   
5   복리후생 주5일근무, 연차, 월차, 보건휴가, 경조휴가, 결혼기념일 휴가, 국민연금...   
6   복리후생 주5일근무, 연차, 보건휴가, 반차, 육아휴직, 남성출산휴가, 산전/후 휴...   
7   복리후생 주5일근무, 연차, 월차, 경조휴가, 국민연금, 의료보험, 산재보험, 고용...   
8        주소  (06797) 서울특별시 서초구 헌릉로 12 (양재동) 지도보기  ...   
9   복리후생 주5일근무, 국민연금, 의료보험, 산재보험, 고용보험, 인센티브, 우수사원...   
10  복리후생 주5일근무, 연차, 월차, 보건휴가, 반차, 국민연금, 의료보험, 산재보험...   
11  복리후생 주5일근무, - 휴양 콘도 운영 : 대명리조트\r\n- 자기계발비 : 어학...   
12  복리후생 주5일근무, 국민연금, 의료보험, 산재보험, 고용보험, 인센티브, 우수사원...   
13  복리후생 주5일근무, 연차, 월차, 보건휴가, 반차, 국민연금, 의료보험, 산재보험...   
14  복리후생 주5일근무, 국민연금, 의료보험, 산재보험, 고용보험, 인센티브, 우수사원...   
15  복리후생 연차, 월차, 국민연금, 의료보험, 산재보험, 고용보험, 퇴직금, 간식제공...   
16  복리후생 주5일근무, 연차, 월차, 보건휴가, 경

In [18]:
robot_df.drop(columns=['주소'],inplace=True)

In [22]:
robot_df.to_csv('../../data/eda_project_data/robot1.csv', sep=',',encoding='utf-8')

In [4]:
import pandas as pd 
robot_df = pd.read_csv('../../data/eda_project_data/robot1.csv', encoding='utf-8')
robot_df

,Unnamed: 0,기업이름,모집명,링크,담당업무,연봉,경력,키워드,링크2,추출된_주소
0,0,(주)잡뉴스솔로몬서치,"로봇 제어 엔지니어 (경력 10년이상) - IT 서비스 전문기업, 그룹 계열사",https://job.incruit.com/jobdb_info/jobpost.asp...,기계·금속·재료,NaN,경력10년↑,로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,"(06703) 서울특별시 서초구 방배로 39 1층 104호 (방배동,미주빌딩) 지도..."
1,1,(주)잡뉴스솔로몬서치,"R&D, 제어 알고리즘 (대리-과장급) - 산업용 로봇 생산기업, 그룹계열사",https://job.incruit.com/jobdb_info/jobpost.asp...,응용프로그래머,NaN,경력3년↑,로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,"(06703) 서울특별시 서초구 방배로 39 1층 104호 (방배동,미주빌딩) 지도..."
2,2,(주)피플케어코리아,자동차 차체설비 전기제어 로봇팀 팀장(부장급) - 2명,https://job.incruit.com/jobdb_info/jobpost.asp...,"전자·반도체, 기계·금속·재료",NaN,경력13~20년,로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,"(06164) 서울특별시 강남구 영동대로 511 (삼성동,트레이드타워) 27층, 3..."
3,3,메리텍산업,자동화기기 전기제어 신입/경력(다관절로봇 티칭 우대) 모집,https://job.incruit.com/jobdb_info/jobpost.asp...,전자·반도체,NaN,신입/경력(연차무관),로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,(15418) 경기도 안산시 단원구 지원로115번길 23 (성곡동) 지도보기 ...
4,4,㈜에이트테크,"[IT/로봇] (주)에이트테크 AI, 로봇제어, 백엔드 개발 채용",https://job.incruit.com/jobdb_info/jobpost.asp...,"빅데이터·AI(인공지능), 소프트웨어·하드웨어, 전기·전자·통신기능",NaN,신입/경력(연차무관),로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,"(22689) 인천광역시 서구 정서진로 410 D동 230호 (오류동,환경산업연구단..."
5,5,(주)프로매치코리아,반도체-로봇기업 컨트롤러 제어SW 개발/기구설계-IMP(임플란타)개발 과장-이사급 6명,https://job.incruit.com/jobdb_info/jobpost.asp...,"전자·반도체, 기계·금속·재료, 응용프로그래머",NaN,경력(연차무관),로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,"(06150) 서울특별시 강남구 테헤란로 333 10층 (역삼동,신도벤처타워) 지도..."
6,6,(주)엑사로보틱스,(주)엑사로보틱스 로봇 소프트웨어 개발 외 신입/경력 정규직 채용,https://job.incruit.com/jobdb_info/jobpost.asp...,응용프로그래머,NaN,신입/경력(연차무관),로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,"(06192) 서울특별시 강남구 테헤란로 406 B동 203호 (대치동,샹제리제센터..."
7,7,(주)유비소시어스,"코스닥상장 중견기업, 수술로봇 개발팀장 모집",https://job.incruit.com/jobdb_info/jobpost.asp...,광학·의료장비,NaN,경력10년↑,로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,"(08505) 서울특별시 금천구 가산디지털2로 123 월드메르디앙2차 1110호, ..."
8,8,현대자동차(주),Robotics Engineer [ 서비스 로봇 제어 ],https://job.incruit.com/jobdb_info/jobpost.asp...,"소프트웨어·하드웨어, 데이터베이스·DBA, 시스템프로그래머",NaN,경력3년↑,로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,(06797) 서울특별시 서초구 헌릉로 12 (양재동) 지도보기 홈페이지 h...
9,9,(주)피플스카우트,[대기업/그룹사]로보틱스&제어 엔지니어/대리-차장급,https://job.incruit.com/jobdb_info/jobpost.asp...,"응용프로그래머, 시스템프로그래머",NaN,경력3~15년,로봇제어,https://job.incruit.com/s_common/jobpost/jobpo...,(06131) 서울특별시 마포구 삼개로 16 근신빌딩 2신관 501호 (도화동) 지...


In [6]:
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm 
from user_agent import generate_user_agent

list3 = []

for idx, row in tqdm(robot_df.iterrows(), total=len(robot_df.index)):
    try:
        req = Request(url=row['링크2'], headers={'User-Agent':generate_user_agent()})
        page = urlopen(req)
        soup = BeautifulSoup(page, 'html.parser')

        tbody = soup.get_text()
        list3.append(tbody)
    except:
        list3.append(None)

100%|██████████| 44/44 [00:06<00:00,  6.95it/s]


In [7]:
list3

['\n\n\n로봇 제어 엔지니어 (경력 10년이상) - IT 서비스 전문기업, 그룹 계열사 - 인크루트 채용\n\n\n\n\n\n\n\n\n\n\n\n\n\n[의뢰사]- IT 서비스 전문기업, 그룹 계열사- 한국타이어 그룹,\xa0 IT 서비스 및 물류 안정화 지원 종합엔지니링 기업-\xa0회사위치/\xa0근무지\xa0: 경기도 판교\xa0[채용내용]1.\xa0채용부문\xa0:\xa0\xa0\xa0\xa0\xa0- 로봇 제어 엔지니어2.\xa0담당업무\xa0:\xa0\xa0\xa0\xa0- 로봇 제어 프로그래밍\xa0 \xa0 \xa0 \xa0* 수직 다관절 로봇/ 물류 이송 시스템 등\xa0 \xa0 - 모터 제어 알고리즘 설계 및 개발\xa0 \xa0 - ROS 기반 각종 센서 장치 드라이버 개발\xa0 \xa0 - 물류 자동화 시스템 구축 및 개발3.\xa0채용직급\xa0:\xa0\xa0\xa0\xa0- 경력에 따라 결정4.\xa0자격요건\xa0:\xa0\xa0\xa0\xa0- 4년제\xa0대졸\xa0이상 및 메카트로닉스, 로봇, 제어, 전자 등 관련 전공자 (석사 이상 우대)\xa0\xa0 \xa0 - 관련경력 10년이상\xa0 \xa0 -\xa0우대사항 :\xa0 \xa0 \xa0 \xa0* Robotics 분야 개발에 능통한 자\xa0 \xa0 \xa0 \xa0* Embedded System 구성 및 소프트웨어 개발 경력자\xa0 \xa0 \xa0 \xa0* ROS/ ROS2 이용 로봇 SW 개발 경험자\xa0 \xa0 \xa0\xa0\xa0 \xa0 \xa0\xa05.\xa0회사위치/\xa0근무지\xa0:\xa0\xa0\xa0\xa0- 경기도 판교\xa0[전형방법]-\xa0서류전형-\xa0면접전형,\xa0서류전형\xa0합격자에\xa0한하여\xa0개별\xa0통보[제출서류]-\xa0한글이력서\xa0:\xa0사진첨부\xa0및\xa0희망직급,\xa0희망연봉,\xa0연락처\xa0등\xa0기재-\xa0한글경력\xa0및\xa0자기소개서\xa0:\xa0경력\x

In [8]:
robot_df['필요역량'] = list3

In [9]:
robot_df.drop(columns=['링크','링크2'],inplace=True)

In [10]:
import re

# 주소에서 (숫자) 제거하는 함수
def remove_numbers_in_parentheses(address):
    if pd.isna(address):
        return address
    return re.sub(r'\(\d+\)', '', address)

# '주소' 열에 함수 적용
robot_df['추출된_주소'] = robot_df['추출된_주소'].apply(remove_numbers_in_parentheses)

In [11]:
robot_df.drop(columns=['모집명'],inplace=True)

robot_df['추출된_주소'] =  robot_df['추출된_주소'].str.split('지도보기').str[0]
robot_df['추출된_주소'] =  robot_df['추출된_주소'].str.split('홈페이지').str[0]

robot_df.rename(columns={'경력':'고용형태','추출된_주소':'기업주소'},inplace=True)
new_order = ['기업이름', '고용형태', '연봉', '기업주소', '키워드', '담당업무', '필요역량']
drone_df = robot_df[new_order]

In [15]:
robot_df.to_csv('../../data/eda_project_data/Robot_syt.csv', sep=',', encoding='utf-8', index=False)

In [14]:
robot_df.drop(columns=['Unnamed: 0'],inplace=True)

In [16]:
robot_df

,기업이름,담당업무,연봉,고용형태,키워드,기업주소,필요역량
0,(주)잡뉴스솔로몬서치,기계·금속·재료,NaN,경력10년↑,로봇제어,"서울특별시 서초구 방배로 39 1층 104호 (방배동,미주빌딩)","\n\n\n로봇 제어 엔지니어 (경력 10년이상) - IT 서비스 전문기업, 그룹 ..."
1,(주)잡뉴스솔로몬서치,응용프로그래머,NaN,경력3년↑,로봇제어,"서울특별시 서초구 방배로 39 1층 104호 (방배동,미주빌딩)","\n\n\nR&D, 제어 알고리즘 (대리-과장급) - 산업용 로봇 생산기업, 그룹계..."
2,(주)피플케어코리아,"전자·반도체, 기계·금속·재료",NaN,경력13~20년,로봇제어,"서울특별시 강남구 영동대로 511 (삼성동,트레이드타워) 27층, 33층",\n\n\n자동차 차체설비 전기제어 로봇팀 팀장(부장급) - 2명 - 인크루트 채용...
3,메리텍산업,전자·반도체,NaN,신입/경력(연차무관),로봇제어,경기도 안산시 단원구 지원로115번길 23 (성곡동),\n\n\n자동화기기 전기제어 신입/경력(다관절로봇 티칭 우대) 모집 - 인크루트 ...
4,㈜에이트테크,"빅데이터·AI(인공지능), 소프트웨어·하드웨어, 전기·전자·통신기능",NaN,신입/경력(연차무관),로봇제어,"인천광역시 서구 정서진로 410 D동 230호 (오류동,환경산업연구단지 본부동)","\n\n\n[IT/로봇] (주)에이트테크 AI, 로봇제어, 백엔드 개발 채용 - ..."
5,(주)프로매치코리아,"전자·반도체, 기계·금속·재료, 응용프로그래머",NaN,경력(연차무관),로봇제어,"서울특별시 강남구 테헤란로 333 10층 (역삼동,신도벤처타워)",\n\n\n반도체-로봇기업 컨트롤러 제어SW 개발/기구설계-IMP(임플란타)개발 과...
6,(주)엑사로보틱스,응용프로그래머,NaN,신입/경력(연차무관),로봇제어,"서울특별시 강남구 테헤란로 406 B동 203호 (대치동,샹제리제센터)",\n\n\n(주)엑사로보틱스 로봇 소프트웨어 개발 외 신입/경력 정규직 채용 - 인...
7,(주)유비소시어스,광학·의료장비,NaN,경력10년↑,로봇제어,"서울특별시 금천구 가산디지털2로 123 월드메르디앙2차 1110호, 1116호","\n\n\n코스닥상장 중견기업, 수술로봇 개발팀장 모집 - 인크루트 채용\n\n\n..."
8,현대자동차(주),"소프트웨어·하드웨어, 데이터베이스·DBA, 시스템프로그래머",NaN,경력3년↑,로봇제어,서울특별시 서초구 헌릉로 12 (양재동),\n\n\nRobotics Engineer [ 서비스 로봇 제어 ] - 인크루트 채...
9,(주)피플스카우트,"응용프로그래머, 시스템프로그래머",NaN,경력3~15년,로봇제어,서울특별시 마포구 삼개로 16 근신빌딩 2신관 501호 (도화동),\n\n\n[대기업/그룹사]로보틱스&제어 엔지니어/대리-차장급 - 인크루트 채용\n...
